In [ ]:
import cv2
import mediapipe as mp 
from sklearn.cluster import KMeans
import pandas as pd
import mediapipe.python.solutions.drawing_utils as mp_drawing 

mp_pose = mp.solutions.pose

# Configure model complexity and landmarks
mp_pose.Pose(
  static_image_mode=False,
  model_complexity=2,
  smooth_landmarks=True,
  min_detection_confidence=0.5,
  min_tracking_confidence=0.5
)
# Get the video from Webcam
cap = cv2.VideoCapture(0)
landmark_list = []

with mp_pose.Pose() as pose_detector:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      continue

    # Detect landmarks
    results = pose_detector.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)) 
    
    # Get landmark values
    landmarks = results.pose_landmarks.landmark
    landmark_values = [[lmk.x, lmk.y, lmk.z] for lmk in landmarks]

    # Append to list
    landmark_list.append(landmark_values)

    # Draw landmarks
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.imshow('Pose Estimation', image)

    if cv2.waitKey(5) == 27:
      break

# Clustering
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(landmark_list)

print(f"Found {num_clusters} pose clusters")

# Save clusters
df = pd.DataFrame(kmeans.labels_, columns=['cluster'])
df.to_csv('pose_clusters.csv')

cap.release()